[out:json][timeout:25];
// fetch area “Zürich” to search in
{{geocodeArea:Zürich}}->.searchArea;
// gather results
nwr["tourism"="hotel"](area.searchArea);
// print results
out geom;

## Overpass turbo query to get all available hotels in Switzerland

In [7]:
import requests
import json
from pandas import json_normalize
import folium

# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Alternative Overpass query for hotels in Zürich using explicit area definition
query = """
        [out:json][timeout:25];
        area["name"="Zürich"]["boundary"="administrative"]["admin_level"="8"];
        nwr["tourism"="hotel"](area);
        out geom;
        """

# Web API request with error handling
r = requests.get(url, params={'data': query})

# Check if the request was successful
if r.status_code == 200:
    try:
        data = r.json()['elements']
        
        # Save data to file
        with open('hotels_zurich.json', 'w') as json_file:
            json.dump(data, json_file)

        # Store data in DataFrame
        df = json_normalize(data)
        print(df.head(5))

        # Entfernen von Zeilen mit NaN-Werten in den 'lat' und 'lon' Spalten
        locations = locations.dropna(subset=['lat', 'lon'])

        # Karte für Zürich erstellen
        map = folium.Map(location=[locations["lat"].mean(), locations["lon"].mean()],
                        zoom_start=12,
                        control_scale=True)

        # Marker für Hotels hinzufügen
        for i in range(0, len(locations)):
            folium.Marker(
                location=(locations.iloc[i]['lat'], locations.iloc[i]['lon']),
                popup=locations.iloc[i].get('tags.name', 'Hotel ohne Namen')
            ).add_to(map)

        # Karte anzeigen
        map


    except json.JSONDecodeError:
        print("Fehler beim Parsen der JSON-Daten. Die Antwort ist möglicherweise kein gültiges JSON.")
        print("Antwortinhalt:", r.text)  # Zum Debuggen
else:
    print(f"Fehler bei der Anfrage. Statuscode: {r.status_code}")
    print("Antwortinhalt:", r.text)  # Zum Debuggen


   type         id        lat       lon tags.addr:city tags.addr:housenumber  \
0  node  119800304  47.384149  8.547993         Zürich                   101   
1  node  261651357  47.373738  8.553185         Zürich                    26   
2  node  268411423  47.361623  8.551307         Zürich                    63   
3  node  268411928  47.367112  8.539343         Zürich                     1   
4  node  268412177  47.369769  8.539772         Zürich                    12   

  tags.addr:postcode    tags.addr:street                   tags.alt_name  \
0               8006  Universitätstrasse                   Hotel Rigihof   
1               8032      Plattenstrasse                             NaN   
2               8008      Seefeldstrasse                             NaN   
3               8001          Talstrasse                             NaN   
4               8001         Poststrasse  Mandarin Oriental Savoy Zürich   

                 tags.brand  ... tags.payment:jcb tags.payment

## Plot hotels on map

In [9]:
import folium

# Nur Zeilen mit gültigen 'lat' und 'lon'-Werten verwenden
locations = df[["lat", "lon", "tags.alt_name", "tags.brand"]].dropna(subset=['lat', 'lon'])
print(locations.head(5))

# Falls spezifische Hotelketten oder -namen gewünscht sind (z.B. nur bestimmte Marken anzeigen)
locations_hotel_1 = locations[locations["tags.brand"].str.contains("Leonardo Boutique Hotels", na=False)]
locations_hotel_2 = locations[locations["tags.brand"].str.contains("Mandarin Oriental Hotel", na=False)]

# Marker Symbole
url_hotel_1 = 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-green.png'
url_hotel_2 = 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-orange.png'

# Karte erstellen, zentriert auf Zürich
map = folium.Map(location=[locations["lat"].mean(), locations["lon"].mean()],
                 zoom_start=12,
                 control_scale=True)

# Marker für 'Leonardo Boutique Hotels'
for i in range(0, len(locations_hotel_1)):
    folium.Marker(
        location=(locations_hotel_1.iloc[i]['lat'], locations_hotel_1.iloc[i]['lon']),
        popup=locations_hotel_1.iloc[i].get('tags.alt_name', 'Hotel ohne Namen'),
        icon=folium.features.CustomIcon(url_hotel_1, icon_size=(16, 28))
    ).add_to(map)
    
# Marker für 'Mandarin Oriental Hotel'
for i in range(0, len(locations_hotel_2)):
    folium.Marker(
        location=(locations_hotel_2.iloc[i]['lat'], locations_hotel_2.iloc[i]['lon']),
        popup=locations_hotel_2.iloc[i].get('tags.alt_name', 'Hotel ohne Namen'),
        icon=folium.features.CustomIcon(url_hotel_2, icon_size=(16, 28))
    ).add_to(map)

# Karte anzeigen
map


         lat       lon                   tags.alt_name  \
0  47.384149  8.547993                   Hotel Rigihof   
1  47.373738  8.553185                             NaN   
2  47.361623  8.551307                             NaN   
3  47.367112  8.539343                             NaN   
4  47.369769  8.539772  Mandarin Oriental Savoy Zürich   

                 tags.brand  
0  Leonardo Boutique Hotels  
1                       NaN  
2                       NaN  
3                       NaN  
4   Mandarin Oriental Hotel  


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [10]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1025-azure
Datetime: 2024-11-09 18:55:35
Python Version: 3.11.10
-----------------------------------
